# (Distributed) Data Parallel
В этой части курса мы познакомимся с Data Parallel режимом обучения. Посмотрим, как его можно запускать и какие есть проблемы в нем.

## One model Multi GPU

In [12]:
!nvidia-smi

Thu Nov 17 15:08:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.43.04    Driver Version: 515.43.04    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:8D:00.0 Off |                  N/A |
|  0%   55C    P8    47W / 300W |    848MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:C7:00.0 Off |                  N/A |
| 36%   

In [13]:
import torch

In [14]:
torch.zeros((1, 1), device="cuda")

tensor([[0.]], device='cuda:0')

In [19]:
torch.zeros((1, 1), device="cuda:0")

tensor([[0.]], device='cuda:0')

In [20]:
torch.zeros((1, 1), device="cuda:1")

tensor([[0.]], device='cuda:1')

In [22]:
torch.ones((1, 1), device="cuda:0") + torch.ones((1, 1), device="cuda:1").to("cuda:0")

tensor([[2.]], device='cuda:0')

In [29]:
torch.ones((1, 1), device="cuda:0").device

device(type='cuda', index=0)

## DataParallel

Просто добавляем [DataParallel](https://pytorch.org/docs/stable/generated/torch.nn.DataParallel.html)

In [ ]:
torch.nn.parallel.DataParallel(model)

## (Distributed) DataParallel


Несколько шагов: 

1) Инициализация процесса (torch.distributed.init)

2) Инициализация модели (DistributedDataParallel)

3) Подготовка данных (просто, но сложно)

In [ ]:
torch.distributed.init_process_group(backend, rank=args.rank, world_size=2)

In [ ]:
torch.nn.parallel.DistributedDataParallel(model)

## (Distributed) Data Sampler


Посмотрим, что какие данные считывает модель в обычном режиме. И попробуем поменять Sampler на [DistributedSampler](https://pytorch.org/docs/stable/data.html#torch.utils.data.distributed.DistributedSampler).

In [ ]:
torch.utils.data.distributed.DistributedSampler()

## Distributed Synchronization

Обсудим несколько примитивов распределённого обучения.

In [ ]:
torch.distributed.get_rank()

In [ ]:
torch.distributed.get_world_size()

In [ ]:
torch.distributed.barrier()

In [ ]:
torch.distributed.send()
torch.distributed.recv()

In [ ]:
torch.distributed.broadcast()

In [ ]:
torch.distributed.all_reduce()

In [ ]:
torch.distributed.gather()
torch.distributed.scatter()

# Homework

- Скоро на почту вам придет инструкция к подключению к серверу с несколькими видеокартами.
- Подключитесь и проверьте, что всё у вас работает.
- Установите [miniconda](https://conda.io/projects/conda/en/stable/user-guide/install/index.html)
- Установите torch/torchvision/остальные библиотеки для вашего фреймворка
- Добавьте в ваш фреймворк автоматическую работу с torch.distributed:
    - Если нет видеокарты, то обучение происходит на cpu
    - Если есть только одна видеокарта, то обучение происходит в обычном режиме
    - Если есть > 1 видеокарты, то обучение происходит в distributed режиме